# ***MAIN FUNCTIONALITY***

In [1]:
from meteo.Site import Site
from solar.SolarPVArray import SolarPVArray
from solar.SolarPVModel import SolarPVModel

from misc.log_config import configure_logging
from misc.util import load_pv_model

configure_logging()

# Set up site location and get TMY data
name = ""
address = "York St, Belfast, BT15 1ED"
client = "Ulster University"

site = Site(name=name, 
            address=address, 
            client=client, 
            latitude=24.60452, 
            longitude=-5.92860, 
            size=100)

# Set up PV arrays
pv_kwp = 1
surface_pitch = 35
surface_azimuth = -90
lifespan = 25
pv_eol_derating = 0.88
pv_cost = 1250
efficiency = 0.21
temp_coeff = -0.0035
transmission_absorption = 0.9
refraction_index = 0.1

array_1 = SolarPVArray(pv_kwp, surface_pitch, surface_azimuth, lifespan, pv_eol_derating,
                       pv_cost, efficiency, temp_coeff, transmission_absorption, refraction_index)

array_2 = SolarPVArray(1, 35, -45, cost_per_kWp=pv_cost)
array_3 = SolarPVArray(1, 35, 0, 25, 0.88)
array_4 = SolarPVArray(1, 35, 45)
array_5 = SolarPVArray(1, 35, 90)

# Arrange all arrays into list for modelling
arrays = [array_1, array_2, array_3, array_4, array_5]

# Run SolarPV model
pv_model = SolarPVModel(site, arrays)

# Save model as .wmm file
pv_model.save_model()

# Load .wwm model from file
pv_model = load_pv_model()


2024-02-22 01:06:18,736 - INFO - Fetching TMY data for latitude: 24.60452, longitude: -5.9286
2024-02-22 01:06:19,650 - INFO - HTTP Request: GET https://re.jrc.ec.europa.eu/api/tmy?lat=24.60452&lon=-5.9286&startyear=2005&endyear=2015&outputformat=json "HTTP/1.1 200 OK"
2024-02-22 01:06:19,998 - INFO - TMY data obtained for: 24.60452, longitude: -5.9286
2024-02-22 01:06:19,999 - INFO - *******************
2024-02-22 01:06:19,999 - INFO - Solar PV array created: Size: 1kW, Pitch: 35deg, Azimuth: -90deg, Lifespan: 25yrs
2024-02-22 01:06:20,000 - INFO - Solar PV array created: Size: 1kW, Pitch: 35deg, Azimuth: -45deg, Lifespan: 25yrs
2024-02-22 01:06:20,000 - INFO - Solar PV array created: Size: 1kW, Pitch: 35deg, Azimuth: 0deg, Lifespan: 25yrs
2024-02-22 01:06:20,001 - INFO - Solar PV array created: Size: 1kW, Pitch: 35deg, Azimuth: 45deg, Lifespan: 25yrs
2024-02-22 01:06:20,001 - INFO - Solar PV array created: Size: 1kW, Pitch: 35deg, Azimuth: 90deg, Lifespan: 25yrs
2024-02-22 01:06:20,0

0       1.0
1       1.0
2       1.0
3       1.0
4       1.0
       ... 
8755    1.0
8756    1.0
8757    1.0
8758    1.0
8759    1.0
Name: WS10m, Length: 8760, dtype: float64
0       1.0
1       1.0
2       1.0
3       1.0
4       1.0
       ... 
8755    1.0
8756    1.0
8757    1.0
8758    1.0
8759    1.0
Name: WS10m, Length: 8760, dtype: float64
0       1.0
1       1.0
2       1.0
3       1.0
4       1.0
       ... 
8755    1.0
8756    1.0
8757    1.0
8758    1.0
8759    1.0
Name: WS10m, Length: 8760, dtype: float64
0       1.0
1       1.0
2       1.0
3       1.0
4       1.0
       ... 
8755    1.0
8756    1.0
8757    1.0
8758    1.0
8759    1.0
Name: WS10m, Length: 8760, dtype: float64
0       1.0
1       1.0
2       1.0
3       1.0
4       1.0
       ... 
8755    1.0
8756    1.0
8757    1.0
8758    1.0
8759    1.0
Name: WS10m, Length: 8760, dtype: float64


In [2]:
pv_model.summary_grouped.quarterly

,PV_Gen_kWh_Total,E_POA_kWm2_Total,Panel_POA_kWm2_Total,IAM_Loss_kWm2_Total,PV_Thermal_Loss_kWh_Total,E_Beam_kWm2_Total,E_Diffuse_kWm2_Total,E_Ground_kWm2_Total,ET_HRad_kWm2_Total,Cell_Temp_C_Avg,T2m
Month_of_Year,,,,,,,,,,,
1,2508.825,2747.962,2659.240,88.722,148.768,2096.842,603.623,47.497,3580.473,24.989,19.434
2,2737.907,3110.756,3021.246,89.510,278.004,2141.813,905.353,63.590,4943.103,36.926,30.581
3,2532.630,2945.692,2872.911,72.781,331.730,1849.913,1037.009,58.770,4791.754,43.324,37.187
4,2268.333,2520.284,2437.219,83.065,158.724,1865.775,612.282,42.227,3319.790,28.225,22.941


In [3]:
pv_model.summary_grouped.quarterly

,PV_Gen_kWh_Total,E_POA_kWm2_Total,Panel_POA_kWm2_Total,IAM_Loss_kWm2_Total,PV_Thermal_Loss_kWh_Total,E_Beam_kWm2_Total,E_Diffuse_kWm2_Total,E_Ground_kWm2_Total,ET_HRad_kWm2_Total,Cell_Temp_C_Avg,T2m
Month_of_Year,,,,,,,,,,,
1,2515.938,2747.962,2659.240,88.722,141.655,2096.842,603.623,47.497,3580.473,24.708,19.434
2,2746.983,3110.756,3021.246,89.510,268.928,2141.813,905.353,63.590,4943.103,36.573,30.581
3,2544.647,2945.692,2872.911,72.781,319.714,1849.913,1037.009,58.770,4791.754,42.860,37.187
4,2281.845,2520.284,2437.219,83.065,145.211,1865.775,612.282,42.227,3319.790,27.680,22.941


In [3]:
import plotly.express as px
import seaborn as sns

px.bar(pv_model.summary_grouped.daily.PV_Gen_kWh_Total)

# ***Temperature Tests***

In [3]:
import math

def calc_array_temp_homer(
    e_poa,
    ambient_temp,
    cell_temp_coeff=-0.0035,
    electrical_eff=0.21,
    cell_NOCT=42,
    ambient_NOCT=20,
    e_poa_NOCT=800,
    cell_temp_STC=25,
    transmittance_absorptance=0.9,
):
    """Calculates the cell temperature of a PV panel.

    Parameters:
    - e_poa: Plane of array irradiance in kW/m^2.
    - ambient_temp: Ambient temperature in degrees Celsius.
    - cell_temp_coeff: Temperature coefficient of the PV cell.
    - electrical_eff: Electrical efficiency of the PV panel.
    - cell_NOCT, ambient_NOCT: Nominal operating cell temperature and the corresponding ambient temperature.
    - e_poa_NOCT: Irradiance at NOCT conditions in W/m^2.
    - cell_temp_STC: Cell temperature at standard test conditions in degrees Celsius.
    - transmittance_absorptance: Transmittance and absorptance product of the PV panel.

    Returns:
    - Cell temperature of the PV panel.
    """
    temp_factor = (cell_NOCT - ambient_NOCT) * ((e_poa * 1000) / e_poa_NOCT)
    numerator = ambient_temp + temp_factor * (
        1
        - (electrical_eff * (1 - cell_temp_coeff * cell_temp_STC))
        / transmittance_absorptance
    )
    denominator = 1 + temp_factor * (
        cell_temp_coeff * electrical_eff / transmittance_absorptance
    )

    return numerator / denominator


def calc_array_temp_pvsyst(e_poa, ambient_temp, windspeed, uc=29, uv=1.2):
    u = uc + (uv * windspeed)
    tcell = ambient_temp + (1/u) * (0.9 * e_poa * 1000 * (1-0.21))
    return tcell

def calc_array_temp_sandia(e_poa: float, ambient_temp: float, wind_speed: float, 
                           a: float = -3.47, b: float = -0.0594) -> float:
    """
    Calculate the temperature of a photovoltaic (PV) array based on the Sandia method.
    
    Parameters:
    - e_poa (float): Plane of array irradiance in kW/m^2. Represents the solar irradiance incident on the PV array.
    - ambient_temp (float): Ambient temperature in degrees Celsius.
    - wind_speed (float): Wind speed in m/s at the site of the PV array.
    - a (float): Coefficient a in the exponential model, defaulting to -3.47.
    - b (float): Coefficient b in the exponential model, defaulting to -0.0594.
    
    Returns:
    - float: Estimated temperature of the PV array in degrees Celsius.
    """
    array_temp = e_poa * 1000 * math.exp(a + b * wind_speed) + ambient_temp
    return array_temp

def calc_array_temp_faiman(e_poa, ambient_temp, wind_speed, U_0=25, U_1=6.84):
    array_temp = ambient_temp + (e_poa * 1000 / (U_0 + U_1 * wind_speed))
    return array_temp


# Example usage:
e_poa=0.5
ambient_temp=25
wind_speed=1

list_1 = []
list_2 = []
list_3 = []
list_4 = []
list_5 = []


print("")
wind_speed=10
for i in range(1):
    cell_temperature = calc_array_temp_homer(e_poa, ambient_temp)
    print(f"The cell temperature is {cell_temperature:.2f}°C from the Homer model with {wind_speed}m/s windspeed")
    list_2.append(cell_temperature)
    wind_speed += 1

print("")
wind_speed=0
for i in range(21):
    cell_temperature = calc_array_temp_pvsyst(e_poa, ambient_temp, wind_speed)
    print(f"The cell temperature is {cell_temperature:.2f}°C from the PVSyst model with {wind_speed}m/s windspeed")
    list_3.append(cell_temperature)
    wind_speed += 1

print("")
wind_speed=0
for i in range(21):
    cell_temperature = calc_array_temp_sandia(e_poa, ambient_temp, wind_speed)
    print(f"The cell temperature is {cell_temperature:.2f}°C from the Sandia model with {wind_speed}m/s windspeed")
    list_4.append(cell_temperature)
    wind_speed += 1

print("")
wind_speed=0
for i in range(21):
    cell_temperature = calc_array_temp_faiman(e_poa, ambient_temp, wind_speed)
    print(f"The cell temperature is {cell_temperature:.2f}°C from the Faiman model with {wind_speed}m/s windspeed")
    list_5.append(cell_temperature)
    wind_speed += 1



The cell temperature is 35.66°C from the Homer model with 10m/s windspeed

The cell temperature is 37.26°C from the PVSyst model with 0m/s windspeed
The cell temperature is 36.77°C from the PVSyst model with 1m/s windspeed
The cell temperature is 36.32°C from the PVSyst model with 2m/s windspeed
The cell temperature is 35.90°C from the PVSyst model with 3m/s windspeed
The cell temperature is 35.52°C from the PVSyst model with 4m/s windspeed
The cell temperature is 35.16°C from the PVSyst model with 5m/s windspeed
The cell temperature is 34.82°C from the PVSyst model with 6m/s windspeed
The cell temperature is 34.51°C from the PVSyst model with 7m/s windspeed
The cell temperature is 34.21°C from the PVSyst model with 8m/s windspeed
The cell temperature is 33.93°C from the PVSyst model with 9m/s windspeed
The cell temperature is 33.67°C from the PVSyst model with 10m/s windspeed
The cell temperature is 33.42°C from the PVSyst model with 11m/s windspeed
The cell temperature is 33.19°C fr

In [4]:
%%timeit
cell_temperature = calc_array_temp_faiman(e_poa, ambient_temp, wind_speed)

189 ns ± 0.434 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [5]:
%%timeit
cell_temperature = calc_array_temp_sandia(e_poa, ambient_temp, wind_speed)

225 ns ± 0.518 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [6]:
%%timeit
cell_temperature = calc_array_temp_pvsyst(e_poa, ambient_temp, wind_speed)

257 ns ± 1.86 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [7]:
%%timeit
cell_temperature = calc_array_temp_homer(e_poa, ambient_temp)

429 ns ± 5.61 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [ ]:
import numpy as np
import plotly.express as px

# Define the modified logistic function
def calc_low_light_losses(L, I, k=0.02, I_0=250):
    """
    Modified logistic function to calculate efficiency based on irradiance,
    with a minimum efficiency level.

    Parameters:
    L (float): The rated solar PV size (kWp).
    I (float): The irradiance incident on array (W/m2).
    k (float): The steepness of the curve.
    I_0 (float): The irradiance at which the efficiency is at its midpoint.
    L_min (float): The minimum efficiency level at 10% STC or below.

    Returns:
    float: The calculated efficiency at the given irradiance.
    """
    L_min = L * 0.8125
    I_adj = I / 1000
    eff = L_min + (L - L_min) / (1 + np.exp(-k * (I - I_0)))
    return eff


# Generate a range of irradiance values from 0 to 1000 W/m2 and set L to 1 kWp
L = 1
irradiance_range = np.linspace(0, 1000, 1001)
efficiencies = calc_low_light_losses(L, irradiance_range)

# Plot Graph of results
fig = px.line(efficiencies[:], y=efficiencies[:], x = irradiance_range[:])

fig.update_layout(xaxis_title="Irradiation (W/m2)", yaxis_title="kWp Output",
                  title = "Line graph showing PV kWp output due to low irradiance losses")

# ***Performance Test***

In [ ]:
# Performance Testing
# import pstats
# import cProfile
# from meteo.Site import Site
# from solar.SolarPVArray import SolarPVArray
# from solar.SolarPVModel import SolarPVModel
# from misc.log_config import configure_logging
# from misc.util import load_pv_model
# configure_logging()
# array_1 = SolarPVArray(1, 35, 0, 25, 0.88)
# cProfile.run("SolarPVModel(site=Site(latitude=54.60452, longitude=-5.92860, size=100), arrays=[array_1])", 'performance_test.prof')
# p = pstats.Stats('performance_test.prof')
# p.sort_stats('tottime').print_stats()
# p.sort_stats('cumtime').print_stats()

# ***Example Running Model with variable inputs***

In [ ]:
# %%timeit

from meteo.Site import Site
from solar.SolarPVArray import SolarPVArray
from solar.SolarPVModel import SolarPVModel

from misc.log_config import configure_logging

configure_logging()

# Set up site and obtain TMY data
site = Site(name="Test Site", address = "123 Test Street", client = "Big Gerry", 
            latitude=54.60452, longitude=-5.92860, tmz_hrs_east = 0)

from meteo.Site import Site
from solar.SolarPVArray import SolarPVArray
from solar.SolarPVModel import SolarPVModel

from misc.log_config import configure_logging

configure_logging()

# Set up site and obtain TMY data
site = Site(name="Test Site", address = "123 Test Street", client = "Big Gerry", 
            latitude=54.60452, longitude=-5.92860, tmz_hrs_east = 0)

In [ ]:
# %%timeit

pv_kwp = 1
surface_pitch = 35
surface_azimuth = -90
lifespan = 25
pv_eol_derating = 0.88

# Set up PV arrays
array_1 = SolarPVArray(pv_kwp, surface_pitch, surface_azimuth, lifespan, pv_eol_derating)
array_2 = SolarPVArray(1, 35, -45, 25, 0.88)
array_3 = SolarPVArray(1, 35, 0, 25, 0.88)
array_4 = SolarPVArray(1, 35, 45, 25, 0.88)
array_5 = SolarPVArray(1, 35, 90, 25, 0.88)
arrays = [array_1, array_2, array_3, array_4, array_5]

# Set up and run model
pv_model = SolarPVModel(site=site, arrays=arrays)

In [ ]:
import plotly.express as px

px.bar(pv_model.summary_grouped.hourly[["PV_Gen_kWh_Total", "IAM_Loss_kWm2_Total", "PV_Thermal_Loss_kWh_Total"]])

In [ ]:
px.bar(pv_model.summary_grouped.monthly[["E_Beam_kWm2_Total", "E_Diffuse_kWm2_Total", "E_Ground_kWm2_Total"]])

In [ ]:
px.line(pv_model.models[0]["model_result"][["Cell_Temp_C", "T2m"]])

In [ ]:
px.line(pv_model.summary_grouped.monthly[["ET_HRad_kWm2_Total", "E_POA_kWm2_Total", "Panel_POA_kWm2_Total"]])

In [ ]:
pv_model.summary

In [ ]:
pv_model.combined_model

# ***Visualisation***

In [ ]:
import plotly.express as px
import seaborn as sns

In [ ]:
pv_model.array_model(1).head()

In [ ]:
print(f"PV Gen: {round(pv_model.all_models.PV_Gen_kWh_Array_1.sum(), 3)}kWh")
print(f"E POA: {round(pv_model.all_models.E_POA_kWm2_Array_1.sum(), 3)}kWhm2")

In [ ]:
px.bar(pv_model.summary_grouped.hourly[["PV_Gen_kWh_Total", "IAM_Loss_kWm2_Total", "PV_Thermal_Loss_kWh_Total"]])

In [ ]:
px.bar(pv_model.summary_grouped.monthly[["PV_Gen_kWh_Total", "IAM_Loss_kWm2_Total", "PV_Thermal_Loss_kWh_Total"]])

In [ ]:
px.bar(pv_model.summary_grouped.weekly[["PV_Gen_kWh_Total", "IAM_Loss_kWm2_Total", "PV_Thermal_Loss_kWh_Total"]])

In [ ]:
px.bar(pv_model.summary_grouped.daily[["PV_Gen_kWh_Total", "IAM_Loss_kWm2_Total", "PV_Thermal_Loss_kWh_Total"]])

In [ ]:
px.line(pv_model.models[0]["model_result"][["Cell_Temp_C", "T2m"]])


In [ ]:
sns.lineplot(pv_model.models[0]["model_result"][["Cell_Temp_C", "T2m"]])

In [ ]:
px.line(pv_model.models[0]["model_result"][["ET_HRad_kWm2", "E_POA_kWm2"]])

In [ ]:
sns.lineplot(pv_model.models[0]["model_result"][["ET_HRad_kWm2", "E_POA_kWm2"]])

# ***Mass Model Testing***

In [ ]:
# %%timeit
import random

# Assuming the Site, SolarPVArray, and SolarPVModel classes are defined elsewhere
site = Site(name="Test Site", address="123 Test Street", client="Big Gerry", 
            latitude=54.60452, longitude=-5.92860, tmz_hrs_east=0)

surface_pitch = 35
azimuth_list = [90, 45, 0, -45, -90]
pv_eol = 0.88
cost = 1250
lifespan = 25
eff = 0.21
temp_coeff = -0.0048
t_ab = 0.9

max_pv = 7.5
min_pv = 2.5
pv_step_size = 0.25
pv_list = []
pv_models = {}

for i in range(int((max_pv - min_pv) / pv_step_size)):
    pv_list.append(min_pv + (i * pv_step_size))

for pv_size in pv_list:
    pv_kwp = pv_size

    array_1 = SolarPVArray(pv_kwp, surface_pitch, azimuth_list[0], lifespan, pv_eol, cost, eff, temp_coeff, t_ab)
    array_2 = SolarPVArray(pv_kwp, surface_pitch, azimuth_list[1], lifespan, pv_eol, cost, eff, temp_coeff, t_ab)
    array_3 = SolarPVArray(pv_kwp, surface_pitch, azimuth_list[2], lifespan, pv_eol, cost, eff, temp_coeff, t_ab)
    array_4 = SolarPVArray(pv_kwp, surface_pitch, azimuth_list[3], lifespan, pv_eol, cost, eff, temp_coeff, t_ab)
    array_5 = SolarPVArray(pv_kwp, surface_pitch, azimuth_list[4], lifespan, pv_eol, cost, eff, temp_coeff, t_ab)

    # Initialize and run the SolarPVModel with all arrays
    arrays = [array_1, array_2, array_3, array_4, array_5]
    pv_model = SolarPVModel(site=site, arrays=arrays)

    pv_models[pv_size] = pv_model  # Store the model with pv_size as the key

Total_gen = 0

for model in pv_models.values():
    Total_gen += model.summary.PV_Gen_kWh_Annual

print("")
print("***************************************************")
print(f"{round(Total_gen / 1000, 3)} MWh from {len(pv_list)} models simulating {len(pv_list) * 5} PV arrays in total.")


In [ ]:
pv_models.get(2.5).models[0]["model_result"]

In [ ]:
# Access values from dictionary
pv_models.get(2.5).summary_grouped.daily.PV_Gen_kWh_Total.sum()

In [ ]:
sns.barplot(pv_models.get(2.5).summary_grouped.monthly.Cell_Temp_C_Avg - pv_models.get(2.5).summary_grouped.monthly.T2m)

In [ ]:
sns.barplot(pv_models.get(2.5).summary_grouped.weekly.PV_Gen_kWh_Total)

In [ ]:
sns.barplot(data=pv_models.get(2.5).summary_grouped.daily, x="Day_of_Year", y="PV_Gen_kWh_Total", 
            hue="PV_Gen_kWh_Total", palette="viridis")

In [ ]:
for pv_size in pv_list:
    print(f"{round(pv_models.get(pv_size).summary_grouped.daily.PV_Gen_kWh_Total.sum() / 1000, 3)} MWh/yr")

# ***VISUAL TESTING***

In [ ]:
variables = ['E_Beam_Wm2', 'E_Diffuse_Wm2', 'E_Ground_Wm2', 'E_POA_Wm2', 'ET_HRad_Wm2', 
            'Cell_Temp_C', 'PV_Gen_kWh', 'AOI', 'Zenith_Angle']

testy = pv_model.all_models

variable = "PV_Gen_kWh"

variable_1 = f"{variable}_Array_1"
variable_2 = f"{variable}_Array_2"
variable_3 = f"{variable}_Array_3"
variable_4 = f"{variable}_Array_4"
variable_7 = f"{variable}_Total"
variable_8 = f"{variable}_Avg"

if variable_1 in testy:
    sns.lineplot(testy.groupby("Hour_of_Day")[variable_1].mean())
if variable_2 in testy:
    sns.lineplot(testy.groupby("Hour_of_Day")[variable_2].mean())
if variable_3 in testy:
    sns.lineplot(testy.groupby("Hour_of_Day")[variable_3].mean())
if variable_4 in testy:
    sns.lineplot(testy.groupby("Hour_of_Day")[variable_4].mean())
if variable_7 in testy:
    sns.lineplot(testy.groupby("Hour_of_Day")[variable_7].mean(), alpha=0.4, ls="--")
if variable_8 in testy:
    sns.lineplot(testy.groupby("Hour_of_Day")[variable_8].mean(), alpha=0.4, ls="--")

In [ ]:
sns.lineplot(pv_model.combined_model["Cell_Temp_C_Avg"], alpha = 0.2, ls = "--", color = "orange")
sns.lineplot(pv_model.combined_model["T2m"])

In [ ]:
sns.barplot(pv_model.summary_grouped.daily.T2m)

In [ ]:
sns.lineplot(pv_model.combined_model.groupby("Hour_of_Day")["PV_Gen_kWh_Total"].mean())

In [ ]:
pv_model.all_models["PV_Gen_kWh_Total"].sum()

In [ ]:
pv_model.all_models.columns

In [ ]:
sns.lineplot(pv_model.all_models["PV_Thermal_Loss_kWh_Total"])

In [ ]:
temp_losses = (pv_model.all_models["PV_Thermal_Loss_kWh_Total"].sum() / pv_model.all_models["PV_Gen_kWh_Total"].sum() * 100)

print(f"{round(temp_losses, 3)}% change to power due to temperature losses")

In [ ]:
pv_model.models[0]["model_result"]

In [ ]:
pv_model.all_models

In [ ]:
pv_model.combined_model

In [ ]:
pv_model.summary

In [ ]:
pv_model.summary_grouped.daily

In [ ]:
pv_model.summary

In [ ]:
pv_model.summary_grouped.monthly

# ***MODEL ANALYSIS***